In [2]:
import pandas as pd
import numpy as np
import warnings

pd.options.mode.chained_assignment = None

warnings.simplefilter(action='ignore', category=FutureWarning)

item_master_excel_file = '../data/inzi/Item Master_08_18_2021 0317.xls'
material_history_excel_file = '../data/inzi/Material Movement History_08_19_2021 1026.xls'
out_put_file = '../output/9-15.xlsx'

item_master = pd.read_excel(
    item_master_excel_file, index_col=False, header=1
)
material_history = pd.read_excel(
    material_history_excel_file, index_col=False, header=1
)

# Item master excel file must have 3 columns 'Material', 'Material Type', 'Procurement'
temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

'''
PREPARE FOR STEP ONE
'''
step_one = pd.merge(material_history, temp_item_master,
                    on="Material", how="left")

step_one.rename(columns={
    'Reference': 'Order Category',
    'Unnamed: 13': 'Order Data',
    'Unnamed: 14': 'Master Category',
    'Unnamed: 15': 'Master Data',
    'Unnamed: 16': 'Remark'
}, inplace=True)

step_one = step_one.iloc[1:, :]

# pd.set_option('mode.chained_assignment', None)

step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

cols = step_one.columns.tolist()

cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

step_one = step_one[cols]

step_one['Account code'] = [int(x)
                            for x in (step_one['Movement Type'].str[:3])]

cols = step_one.columns.tolist()
cols = cols[:7] + [cols[-1]] + cols[7: -1]
step_one = step_one[cols]

columns_name = {
    'H': 'Account code',
    'p': 'Order Category',
    'W': 'Material Type',
    'X': 'Procurement'
}



In [6]:
summary = step_one.pivot_table(index=["Material"],
                            columns="Account code", values="Quantity", aggfunc='sum')


summary.fillna(0, inplace=True)
in_columns = []
out_columns = []


for column in summary.columns:
    if column in [101, 602, 720, 801, 809]:
        in_columns.append(column)
    if column in [102, 201, 261, 555, 601, 609, 721, 803]:
        out_columns.append(column)

summary['IN'] = summary[in_columns].sum(axis=1)
summary['OUT'] = summary[out_columns].sum(axis=1)

In [3]:
for column in pd.unique(step_one['Account code']):
    print(column)

101
261
401
343
609
601
720
344
721
201
623
321
